<a href="https://colab.research.google.com/github/omk42/Pattern-Mining/blob/master/mining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Libraries**

In [0]:
import numpy as np
import matplotlib.pyplot as plt 
from pathlib import Path

**Configurations**

In [0]:
ROOT = '/content/drive/My Drive/MDPM/'

conf_dataset = 'shoes'
conf_imgDir = ROOT + 'data/' + conf_dataset
conf_numClasses = 1
conf_numSamples = 100
# conf_cnnFeatures = ROOT + 'output/' + conf_dataset + '/cnnFeatures.npy'
conf_cnnFeatures = './MDPM_python/cnnFeatures.npy'

# conf_imageDict = ROOT + 'output/' + conf_dataset + '/imageDict.pkl'
# conf_indexImg = ROOT + 'output/' + conf_dataset + '/indexImg.pkl'

# conf_patchSize = 128
# conf_patchSizeL2 = 160
# conf_patchSizeL3 = 192
# conf_stepSize = 32

conf_numTopActivation = 20
conf_supp = 0.01
conf_minLength = 3
conf_maxLength = 6
conf_confid = 30
# conf_invertFile = ROOT + 'output/' + conf_dataset + '/invertFile.npy'
conf_invertFile = './MDPM_python/invertFile.npy'
# conf_inputFile = ROOT + 'output/' + conf_dataset + '/transFile.txt'
conf_inputFile = './MDPM_python/transFile.txt'
# conf_appFile = ROOT + 'mining/appFile.txt'
# conf_outputFile = ROOT + 'output/' + conf_dataset + '/rule.txt'

CROPPED_DIM = 227
IMAGE_DIM = 256

# labels = [x for x in Path(conf_imgDir).iterdir()]

In [0]:
def getInvertFileCls(classID):
  with open (conf_cnnFeatures, 'rb') as f:
    label_cnnFea = np.load (f)
  
  feaSorted = -np.sort(-label_cnnFea, axis=0)
  feaIndex = np.argsort(-label_cnnFea, axis=0)
  invertFile = np.zeros(label_cnnFea.shape)
  
  for i in range(invertFile.shape[1]):
    invertFile[feaIndex[:conf_numTopActivation,i].T, i] = 1
  # print (label_cnnFea, invertFile)
  
  with open(conf_invertFile, 'wb') as f:
    np.save(f, invertFile)

  pos = '4097'
  with open(conf_inputFile,'w') as f:
    for i in range(invertFile.shape[1]):
      for j in np.nonzero(invertFile[:,i])[0]:
        f.write(" "+str(j)) 
      f.write(" "+pos)
      f.write("\n")  

In [5]:
for i in range(1,conf_numClasses+1):
  print ("Combining level 1 CNN feature of class ", i)
  getInvertFileCls(i)

Combining level 1 CNN feature of class  1


In [14]:
%%shell
cd "/content/drive/My Drive/MDPM/mining/apriori/apriori/src"
# chmod 755 "apriori"
./apriori -tr -s0.075 -m8 -n10 -q2 -c30 -R"/content/drive/My Drive/MDPM/mining/appFile.txt" "/content/drive/My Drive/MDPM/output/shoes/transFile.txt" "/content/drive/My Drive/MDPM/output/shoes/rulep075m8n10q2.txt"

./apriori - find frequent item sets with the apriori algorithm
version 6.28 (2019.03.31)        (c) 1996-2019   Christian Borgelt
reading /content/drive/My Drive/MDPM/mining/appFile.txt ... [1 item(s)] done [0.00s].
reading /content/drive/My Drive/MDPM/output/shoes/transFile.txt ... [3959 item(s), 257688 transaction(s)] done [0.36s].
filtering, sorting and recoding items ... [2982 item(s)] done [0.01s].
sorting and reducing transactions ... [231240/257688 transaction(s)] done [0.16s].
building transaction tree ... [351761 node(s)] done [0.05s].
checking subsets of size 1 2 3 4 5 6 7 8 9 10 done [42.57s].
writing /content/drive/My Drive/MDPM/output/shoes/rulep075m8n10q2.txt ... [141 rule(s)] done [0.66s].


Test for InvertFile



In [20]:
conf_cnnFeatures = ROOT + 'scratch/test_arr.npy'
conf_inputFile = ROOT + 'scratch/transFile.txt'
conf_numTopActivation = 4

test_arr = np.array ([[0,34,23,56,23,9,3,21,56,7],[3,66,84,235,6778,324,74,2,1,567],[3,5,63,45,23,56,323,565,43,7]])
test_arr = test_arr.T 
np.save('/content/drive/My Drive/MDPM/scratch/test_arr.npy', test_arr)
print (test_arr)
getInvertFileCls(1)

[[   0    3    3]
 [  34   66    5]
 [  23   84   63]
 [  56  235   45]
 [  23 6778   23]
 [   9  324   56]
 [   3   74  323]
 [  21    2  565]
 [  56    1   43]
 [   7  567    7]]
[[0. 0. 0.]
 [1. 0. 0.]
 [1. 0. 1.]
 [1. 1. 0.]
 [0. 1. 0.]
 [0. 1. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]]


In [0]:
with open('/content/drive/My Drive/MDPM/scratch/test_arr.npy', 'rb') as f:
  t = np.load(f)

assert np.all(t==test_arr)